# Add extra information to DRZ auction results

Query to the open data dataset of the RDW.

It may take a while (10 min) to query all auction results.
- - - - 

### User variables


In [1]:
Date = '2019-06' # yyyy-mm

### Modules and functions

In [2]:
import pandas as pd
import re 
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)

../data/drz-data-2019-06.pkl


### query rdw

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

# make a copy and add info
rdw = drz.copy()


In [5]:
def get_query_url(api_url, regs, token=hidden_api_keys.socrata_apptoken):
    '''construct query url'''
    
    # convert list to string
    reg_list = ''.join(["'{}', ".format(r.replace('-','').upper()) for r in regs])
    reg_list = reg_list[0:-2] # remove trailing ', '
    q = api_url + '?$$app_token=' + token + '&$where='
    # add escaped soql
    q += urllib.parse.quote('kenteken in(' + reg_list + ')')
    return q

Create list of dataframes with different api results

In [ ]:
import urllib

# empty list
dfs = list()
# first element is all registrations
dfs.append(rdw.loc[hasReg,['Reg']])
dfs[0]['kenteken'] = dfs[0].Reg.apply(lambda r: r.replace('-','').upper())
dfs[0].index.name = 'lot_index'
dfs[0] = dfs[0].reset_index().set_index('kenteken')
display(dfs[0].tail())
print(dfs[0].shape)


# do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'
regs = dfs[0].Reg.values
print(len(regs),'registrations in this set')

# query data base
q = get_query_url(api_url,regs)
dfs.append(pd.read_json(q))
dfs[1].set_index('kenteken', inplace=True)
display(dfs[1].tail())
print(dfs[1].shape)

# query other available apis
for api_name in [c for c in dfs[1].columns if c.startswith('api')]:
    print(api_name)
    for api_url in dfs[1][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail())
        print(df0.shape)
        dfs.append(df0)   

In [ ]:
# Merge dataframes from different apis
df = pd.concat(dfs, axis='columns', sort=False)
# add timestamp
df['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df.index.name = 'kenteken'
df = df.reset_index().set_index('lot_index')
display(df.tail())
print(df.shape)

# Merge rdw and drz

In [8]:
rdw = pd.concat([rdw, df.add_prefix('rdw_')], axis='columns', sort=False)

### Saving

In [9]:
file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw.to_pickle(file_name)

../data/rdw-data-2019-06.pkl
